### TD n°1 Deep Learning - Exo 1 FizzBuzz

In [1]:
import numpy as np
import torch

On modélise le pb commme un pb de classification multi-classe.

Les sorties possibles 0 / 1 / 2 / 3 correspondent respectivement 
* au nombre lui-même, 
* à fizz pour les multiples de 3, 
* buzz pour les multiples de 5,
* fizzbuzz pour les multiples de 15.

###  Modélisation sortie de la vérité terrain

In [2]:
def fizz_buzz (i, prediction) :
    return [str(i),..., ...,... ][prediction]

Création de la vérité terrain : [number, fizz, buzz, fizzbuzz]


In [ ]:
def fizz_buzz_encode(i):
    # TODO return 0
    # TODO return 1
    # TODO return 2
    # TODO return 3
    raise NotImplementedError

Testez vos fonctions en vérifiant que vous renvoyez la bonne sortie pour $i=1,\dots,19$

In [ ]:
for i in range(1, 20): 
    print(
        i, 
        fizz_buzz_encode(i),# classe  
        fizz_buzz(i , fizz_buzz_encode(i)) # classe -> résultat i ? fizz ? buzz ? fizzbuzz ?
        )

### Modélisation des entrées et des données

Encodage binaire 


In [ ]:
NUM_DIGITS = 10
def binary_encode (i , num_digits=NUM_DIGITS ) :
    return [... for d in range(num_digits)]

In [ ]:
start_train, end_train = ..., ...
start_test, end_test = ..., ...

# train
X_train = torch.FloatTensor ([ binary_encode (i , NUM_DIGITS ) for i in range (start_train, end_train)])
Y_train = torch.LongTensor ([fizz_buzz_encode(i) for i in range(start_train, end_train)]).squeeze()

# test
X_test = torch.FloatTensor ([ binary_encode (i , NUM_DIGITS) for i in range (start_test, end_test) ])
print(X_test[:5])

### Création du modèle, de la loss et de l'optimiseur

In [ ]:
# nombre de neurones dans la couche cachée
NUM_HIDDEN = 100

# définition du MLP à 1 couche cachée (non linearite ReLU)
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, ...)
    )

__Fonction de coût__ 

CrossEntropyLoss vs NLLLoss ? (negative log likelihood)

CE loss : The input is expected to contain the unnormalized logits for each class, *which do not need to be positive or sum to 1, in general*. 

NLLLoss : il faut avoir calculé un softmax

https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss 

In [7]:
# fonction de coût
loss_fn = torch.nn.CrossEntropyLoss()

In [8]:
# optimiseur --> choix du learning rate
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

### Apprentissage

In [ ]:
BATCH_SIZE = 128

raw_data_test = np.arange(1, 101) # valeurs de test

for epoch in range(10000):
    for start in range(0, len(X_train), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = X_train[start:end]
        batchY = Y_train[start:end]

        # prediction et calcul de la loss
        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)
    
        # mettre les gradients à 0 avant la passe retour (backward)
        optimizer.zero_grad()
    
        # rétro-propagation
        loss.backward()
        optimizer.step()

    # calcul coût  (et affichage)
    loss = loss_fn( model(X_train), Y_train)
    if epoch%100 == 0:
        print('epoch {} training loss {}'.format(epoch, round(loss.item(), 3)))

    # visualisation des résultats en cours d'apprentissage
    if(epoch%1000==0):
        Y_test_pred = model(X_test)
        val, idx = torch.max(Y_test_pred,1)
        ii=idx.data.numpy()
        # numbers = np.arange(1, 101)
        output = np.vectorize(fizz_buzz)(raw_data_test, ii)
        print(output)

__Affichage des résultats__

In [ ]:
# Sortie finale (calcul lisible)
Y_test_pred = model(X_test)
val, idx = torch.max(Y_test_pred,1)
ii=idx.data.numpy()
output = np.vectorize(fizz_buzz)(raw_data_test, ii)
print("============== Final result ============")
print(output)

# Sortie finale (calcul plus compact des predictions)
Y_test_pred = model(X_test)
predictions = zip(range(1, 101), list(Y_test_pred.max(1)[1].data.tolist()))
print("============== Final result ============")
print ([fizz_buzz(i, x) for (i, x) in predictions])

__Calcul des performances (classification accuracy)__

In [ ]:
gtY = np.array([fizz_buzz_encode(i) for i in raw_data_test])
print("test perf: ", np.mean(gtY == ...))

__Avec un validation set !__

In [ ]:
NUM_VAL = 100
p = np.random.permutation(range(len(X_train)))
# permettre de mélanger les nombres
X_train, Y_train = X_train[..., :], Y_train[...]
X_val, Y_val = X_train[..., :], Y_train[...]
X_train, Y_train = X_train[...:, :], Y_train[...]

In [ ]:
BATCH_SIZE = 128
NUM_HIDDEN = 100

model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
    )

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

for epoch in range(10000):
    for start in range(0, len(X_train), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = X_train[start:end]
        batchY = Y_train[start:end]

        # prediction et calcul de la loss
        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)
    
        # mettre les gradients à 0 avant la passe retour (backward)
        optimizer.zero_grad()
    
        # rétro-propagation
        loss.backward()
        optimizer.step()

    # calcul coût  (et affichage)
    loss = loss_fn( model(X_train), Y_train)
    if epoch%100 == 0:
        print('epoch {} training loss {}'.format(epoch, round(loss.item(), 3)))

    # visualisation des résultats en cours d'apprentissage
    # cette fois-ci sur l'ensemble de validation
    if(epoch%1000==0):
        # train acc
        Y_train_pred = model(X_train)
        print("train perf: ", np.mean(Y_train.data.numpy() == Y_train_pred.max(1)[1].data.numpy() ) )
        Y_val_pred = model(X_val)
        print("val perf: ", np.mean(Y_val.data.numpy() == Y_val_pred.max(1)[1].data.numpy() ) )

In [ ]:
# Sortie finale (calcul lisible)
Y_test_pred = model(X_test)
val, idx = torch.max(Y_test_pred,1)
ii=idx.data.numpy()
output = np.vectorize(fizz_buzz)(raw_data_test, ii)
print("============== Final result ============")
print(output)

# Sortie finale (calcul plus compact des predictions)
Y_test_pred = model(X_test)
predictions = zip(range(1, 101), list(Y_test_pred.max(1)[1].data.tolist()))
print("============== Final result ============")
print ([fizz_buzz(i, x) for (i, x) in predictions])

__Experiment with different settings__

* learning rate
* optimizer
* scheduler
* number of training samples
* architecture of the MLP (number of hidden units)
* number of epochs
* ...